=================================================

Objective:  

Tujuan penerapan validasi data menggunakan Great Expectations (GX) adalah untuk menjamin kualitas data streaming musik melalui pemeriksaan kelengkapan, konsistensi format, serta keunikan identitas pengguna, sekaligus mengidentifikasi anomali seperti nilai di luar jangkauan atau ketidakcocokan tipe data. Hasil validasi ini mendukung analisis lanjutan, integrasi dengan Elasticsearch, dan pengambilan keputusan strategis bisnis, sambil menghasilkan dokumentasi otomatis yang mempermudah proses audit.

Latar Belakang:  

Industri streaming musik telah menjadi pendorong utama pertumbuhan pendapatan musik global. Menurut Laporan Musik Global IFPI, pendapatan musik rekaman global meningkat sebesar 10,2% pada tahun 2023 menjadi US$28,6 miliar, menandai tahun kesembilan pertumbuhan berturut-turut. Peningkatan ini terutama didorong oleh pelanggan streaming berbayar, dengan pendapatan streaming berlangganan tumbuh sebesar 11,2% dan menyumbang 48,9% dari pasar global. Untuk pertama kalinya, langganan berbayar melampaui 500 juta, dengan total lebih dari 667 juta pengguna akun berlangganan berbayar di seluruh dunia berdasarkan dari artikel ini: https://www.ifpi.org/ifpi-global-music-report-global-recorded-music-revenues-grew-10-2-in-2023/

=================================================

# 1. Import libraries

In [2]:
import great_expectations as gx
import pandas as pd
import numpy as np

Melakukan import libraries yang dibutuhkan dalam dokumen ini

# 2. Loading dataset

In [3]:
df = pd.read_csv("P2M3_Yoseph_Radityo_data_clean.csv")

Loading dataset yang sudah dibersihkan sebelumnya

# 3. Membuat Timestamp

In [4]:
np.random.seed(42)
timestamps = pd.date_range(start="2023-01-01", periods=len(df), freq="T").strftime("%Y%m%d%H%M")
df["reg_timestamp"] = np.random.choice(timestamps, len(df), replace=False)

In [5]:
df["user_identifier"] = (
    df["country"] + "_" 
    + df["age"].astype(str) + "_" 
    + df["most_played_artist"].str.replace(" ", "-") + "_"
    + df["reg_timestamp"]
)


- Menggunakan random seed untuk menghasilkan timestamp acak untuk repoduktifitas
- Membuat rentang timestamp dari 2023-01-01 dengan interval 1 menit dan menugaskan ke kolom reg_timestamp
- Menciptakan user_identifier unik untuk menggabungkan country, age, most_played_artist dan reg_timestamp

# 4. Setup GX nya

In [6]:
context = gx.get_context()

Melakukan inisiasi Great Expectations menggunakan `gx.get_context()` lalu menyiapkanm lingkunagan untuk validasi data

# 5. Membuat data resources dan data asset

In [7]:
datasource = context.sources.add_or_update_pandas(name="my_pandas_datasource")
data_asset = datasource.add_dataframe_asset(name="my_dataframe_asset", dataframe=df)

Melakukan penambahan sumber data pandas bernama "my_pandas_datasource" dan menciptakan aset data "my_dataframe_asset" dari DataFrame untuk validasi.

# 6. Membangun batch request

In [8]:
batch_request = data_asset.build_batch_request()

Membuat dan membangung permintaan yang akan digunakan pada batch. Dimana dari aset data, akan digunakan untuk menentukan data yang akan divalidasi.

# 7. Membangun expectation Suite

In [9]:
expectation_suite_name = "data_validation_suite"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

{
  "expectation_suite_name": "data_validation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.22"
  }
}

Menciptakan atau memperbarui suite harapan bernama "data_validation_suite" untuk mendefinisikan harapan kualitas data.

# 8. Membuat validator

In [10]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

Menginisialisasi objek validator menggunakan permintaan batch dan suite harapan untuk melakukan validasi.

# 9. Menambahkan Expectation

## 9.1  Unique

In [11]:
validator.expect_column_values_to_be_unique(column="user_identifier")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Memastikan user identifier yang dibuat adalah unique

## 9.2  Age range

In [12]:
validator.expect_column_values_to_be_between(
    column="age", 
    min_value=13, 
    max_value=65
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Memeriksa age antara 13-65

## 9.3  Subscription Type

In [13]:
validator.expect_column_values_to_be_in_set(
    column="subscription_type", 
    value_set=["free", "premium"]
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Memverifikasi 2 jesnis subs yang ada dalam dataset, dimana disini tergbagi menjadi 2 yaitu Free dan Premium

## 9.4  Data Type

In [14]:
validator.expect_column_values_to_be_in_type_list(
    column="discover_engagement", 
    type_list=["float", "float64"]
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Mengkonfirmasi tipe data

## 9.5  Panjang Nama Artis

In [15]:
validator.expect_column_value_lengths_to_be_between(
    column="most_played_artist", 
    min_value=3, 
    max_value=30
)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

memastikan nama panjang artist terdiri dari 3-30 karakter 

## 9.6  Format Grup Umur 

In [16]:
validator.expect_column_values_to_match_regex(
    column="age_group", 
    regex=r"^(<18|18-25|26-35|36-50|50\+)$"
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Lakukan validasi format age group dengan regex

## 9.7  Minimal Lagu yang di Like

In [17]:
validator.expect_column_min_to_be_between(
    column="number_of_songs_liked", 
    min_value=1
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 1
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Memastikan bahwa minimal number of songs liked adalah satu

# 10. Menyimpan expectation suite

In [18]:
validator.save_expectation_suite(discard_failed_expectations=False)

# 11. Membuat dan menjalankan checkpoint

In [ ]:
checkpoint_name = "my_data_validation_checkpoint"
checkpoint = context.add_or_update_checkpoint(
    name=checkpoint_name,
    validator=validator,
)

In [20]:
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/40 [00:00<?, ?it/s]

Menciptkan checckpoint bernama 
checkpoint_name = "my_data_validation_checkpoint"


# 12. Hasil Validasi

In [21]:
print("Checkpoint Result:")
print(checkpoint_result)

Checkpoint Result:
{
  "run_id": {
    "run_name": null,
    "run_time": "2025-07-07T18:32:20.466466+07:00"
  },
  "run_results": {
    "ValidationResultIdentifier::data_validation_suite/__none__/20250707T113220.466466Z/my_pandas_datasource-my_dataframe_asset": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_be_unique",
              "kwargs": {
                "column": "user_identifier",
                "batch_id": "my_pandas_datasource-my_dataframe_asset"
              },
              "meta": {}
            },
            "result": {
              "element_count": 5000,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "missing_count": 0,
              "missing_percent": 0.0,
              "unexpected_percent_total": 0.0,
            

Mencetak hasil checkpoint yang sudah dilakukan

# 13. Meng-generate laporan 

In [22]:
context.build_data_docs()
context.open_data_docs()

Membangun dan membuka dokumentasi data menggunakan Great Expectations, menghasilkan laporan dalam format HTML untuk visualisasi hasil validasi.